## CT5141 Optimisation Assignment 2 (James McDermott)



**Topic** Facility location with real-valued metaheuristic optimisation. How to run experiments.

**Problem** The **facility location** problem is about finding the best locations for some facilities. In our problem, we have a list of cities, and we are going to build some supermarkets to serve the customers in these cities. The supermarkets could be anywhere, not just at the cities, i.e. a supermarket could be near a city or between two cities. There is no population outside the cities. Supermarkets have no limit on capacity. We want to locate the supermarkets so as to minimise the total travel time for all customers. The objective function is formulated in `supermarket.ipynb` with data in `supermarket_cities_locations.csv` and `supermarket_cities_populations.csv`. A plotting function is included. We will implement a Random Search approach to this problem, a Particle Swarm Optimisation approach, and a CMA approach.


In [66]:
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
# Additional import statements required to implement the algorithms
import random
import time
import pyswarms as ps
import itertools
import cma
import pandas as pd

In [51]:
def total_travel_time(x, c, p):
    # x: locations of supermarkets (ie genotype), shape eg (5, 2)
    # c: locations of cities, shape eg (20, 2)
    # p: populations of cities, shape eg (20,)

    # assume that everyone travels to the supermarket exactly once per
    # week

    # assume that travel time is exactly proportional to Euclidean
    # distance from city to supermarket
    
    # calculate all city-supermarket distances. with (eg) 20 cities
    # and 5 supermarkets, we'll have d.shape = (20, 5)
    d = cdist(c, x)
    # everyone in city i will travel to their nearest supermarket,
    # so we need minimum distance from each city to any supermarket.
    dmin = np.min(d, axis=1)
    # there are p[i] people in city i, each travelling a distance
    # dmin[i], so multiply and sum for total travel time
    return np.sum(dmin * p)

def total_travel_time_slow(x, c, p):
    # Notice that the above is vectorised and is only 2-3 lines. 
    # it's equivalent to this slower (easier to understand?) code
    d = cdist(c, x)
    t = 0
    for i, pi in enumerate(p):
        # everyone in city i will travel to their nearest supermarket,
        # so we need minimum distance from city i to any supermarket.
        d = np.min(D[i])
        # there are pi people in city i, each travelling a distance d
        t += pi * d 
    return t


def plot(x, c, p):
    # solutions will be of shape (eg) (10,)
    # but we interpret as shape (5, 2)
    x = x.reshape((-1, 2))
    plt.figure(figsize=(6, 5))
    plt.scatter(c[:, 0], c[:, 1], s=p/10000, label="Cities") # s gives size
    plt.scatter(x[:, 0], x[:, 1], marker="*", label="Supermarkets") # stars for supermarkets
    plt.gca().set_aspect('equal')
    plt.legend()
    plt.savefig("supermarket.pdf")
    plt.show()
    plt.close()


In [52]:
c = np.genfromtxt("supermarket_city_locations.csv")
p = np.genfromtxt("supermarket_city_populations.csv")
ncities = len(c)
nsupermarkets = 5

In [53]:
# common parameters
maxits = 20000
reps = 5

In [54]:
# Random Search
def random_search_algo():
    rs_dict_table = {"Algorithm": "RS", "popsize": "na", "w": "na", "Cg": "na", "Cp": "na"}
    best_cost = 0
    best_distance = list()
    time_spent = list()
    for s in range(0,reps):
        rs_start_time = time.time()
        print("Start Time: ", rs_start_time)
        np.random.seed(s)
        for i in range(maxits):
            supermarkets_loc = np.random.rand(nsupermarkets,2)
            distance = total_travel_time(supermarkets_loc, c, p)
            if i == 0:
                best_cost = distance
            if distance < best_cost:
                best_cost = distance

        rs_end_time = time.time()
        print("End Time: ", rs_end_time)
        best_distance.append(best_cost)
        time_spent.append(end_time - start_time)
        
    rs_dict_table["min"] = min(best_distance)
    rs_dict_table["mean"] = np.mean(best_distance)
    rs_dict_table["stddev"] = np.std(best_distance)
    rs_dict_table["time"] = np.mean(time_spent)
    
    return rs_dict_table

In [55]:
def time_travel_PSO(x):
    travel_time_list = list()
    for loc in x:
        supermarket = loc.reshape(nsupermarkets,2)
        travel_time = total_travel_time(supermarket, c, p)
        travel_time_list.append(travel_time)
    return travel_time_list

In [ ]:
def formulate_pso_dict(pso_dict_value):
    
    
    return dict_table

In [64]:
# Paricle Swarm Optimization: Code referred from the Pyswarm documentation.
#def pso_algo():
c1_list = [1.9,2]
c2_list = [1.7,2]
w_list = [0.9,0.5]
n_particles_list = [80,100]


hyperparameter_list = list()
cost_list = list()
time_list = list()

pso_combinations = list()
for j in range(0, reps):

    np.random.seed(j)

    for c1,c2,w,n in itertools.product(c1_list, c2_list, w_list,n_particles_list):
        pso_start_time = time.time()
        print("Start Time: ", pso_start_time)
        hyperparameters = {"c1": c1, "c2": c2, "w":w}
        optimizer = ps.single.GlobalBestPSO(n_particles=n, dimensions=(nsupermarkets * 2), options=hyperparameters)
        cost, pos = optimizer.optimize(time_travel_PSO, iters=int(maxits/n))
        hyperparameter_tuple = (n, c1, c2, w)
        hyperparameter_list.append(hyperparameter_tuple)
        cost_list.append(cost)

        pso_end_time = time.time()
        print("End Time: ", pso_end_time)
        time_list.append(pso_end_time - pso_start_time)
pso_dict = {"hyperparameter": hyperparameter_list, "cost": cost_list, "time": time_list}
#    return pso_combinations

2021-11-21 01:02:51,969 - pyswarms.single.global_best - INFO - Optimize for 250 iters with {'c1': 1.9, 'c2': 1.7, 'w': 0.9}


Start Time:  1637456571.9615495


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|250/250, best_cost=4.21e+5
2021-11-21 01:02:52,813 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 421016.4805773466, best pos: [-5.34296721  0.45729605  0.63005831  0.40022367  1.43451403 -2.36241105
  0.83288006  0.77272662  0.14345161  0.54639671]
2021-11-21 01:02:52,813 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 1.9, 'c2': 1.7, 'w': 0.9}


End Time:  1637456572.8132386
Start Time:  1637456572.8132386


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|200/200, best_cost=5.14e+5
2021-11-21 01:02:53,588 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 513790.501718807, best pos: [ 0.63943665  0.40915032  0.50980902  0.6333211   0.76051087  0.83970722
 -2.00881197  1.4625664   1.01358906  0.73824222]
2021-11-21 01:02:53,596 - pyswarms.single.global_best - INFO - Optimize for 250 iters with {'c1': 1.9, 'c2': 1.7, 'w': 0.5}


...

End Time:  1637456635.749009
Start Time:  1637456635.749009


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|250/250, best_cost=2.17e+5
2021-11-21 01:03:56,592 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 216501.8768081099, best pos: [0.80954198 0.76829616 0.0861752  0.39393238 0.88256932 0.98920924
 0.12073471 0.6545651  0.64831793 0.40237671]
2021-11-21 01:03:56,600 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 2, 'c2': 2, 'w': 0.5}


End Time:  1637456636.5928154
Start Time:  1637456636.5928154


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|200/200, best_cost=1.71e+5
2021-11-21 01:03:57,360 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 170933.21677323917, best pos: [0.80954182 0.76829712 0.12075346 0.65458087 0.54259911 0.52992585
 0.08617523 0.39393164 0.64831797 0.40237668]


End Time:  1637456637.3603017
{'hyperparameter': [(80, 1.9, 1.7, 0.9), (100, 1.9, 1.7, 0.9), (80, 1.9, 1.7, 0.5), (100, 1.9, 1.7, 0.5), (80, 1.9, 2, 0.9), (100, 1.9, 2, 0.9), (80, 1.9, 2, 0.5), (100, 1.9, 2, 0.5), (80, 2, 1.7, 0.9), (100, 2, 1.7, 0.9), (80, 2, 1.7, 0.5), (100, 2, 1.7, 0.5), (80, 2, 2, 0.9), (100, 2, 2, 0.9), (80, 2, 2, 0.5), (100, 2, 2, 0.5), (80, 1.9, 1.7, 0.9), (100, 1.9, 1.7, 0.9), (80, 1.9, 1.7, 0.5), (100, 1.9, 1.7, 0.5), (80, 1.9, 2, 0.9), (100, 1.9, 2, 0.9), (80, 1.9, 2, 0.5), (100, 1.9, 2, 0.5), (80, 2, 1.7, 0.9), (100, 2, 1.7, 0.9), (80, 2, 1.7, 0.5), (100, 2, 1.7, 0.5), (80, 2, 2, 0.9), (100, 2, 2, 0.9), (80, 2, 2, 0.5), (100, 2, 2, 0.5), (80, 1.9, 1.7, 0.9), (100, 1.9, 1.7, 0.9), (80, 1.9, 1.7, 0.5), (100, 1.9, 1.7, 0.5), (80, 1.9, 2, 0.9), (100, 1.9, 2, 0.9), (80, 1.9, 2, 0.5), (100, 1.9, 2, 0.5), (80, 2, 1.7, 0.9), (100, 2, 1.7, 0.9), (80, 2, 1.7, 0.5), (100, 2, 1.7, 0.5), (80, 2, 2, 0.9), (100, 2, 2, 0.9), (80, 2, 2, 0.5), (100, 2, 2, 0.5), (80, 1.9, 1.7,

In [77]:
dict_table = {"Algorithm": "PSO"}
table_df = pd.DataFrame(pso_dict)
table_df = table_df.sort_values("hyperparameter")

hyperparams = list(table_df["hyperparameter"])
cost_values = list(table_df["cost"])
time_values = list(table_df["time"])

cost_sum = 0
time_sum = 0
compare_hp = 0

cost_sum_dict = {}
time_sum_dict = {}

for val in range(0, len(hyperparams)):
    if val == 0:
        compare_hp == hyperparams[val]
    
    if hyperparams[val] == compare_hp:
        cost_sum = cost_sum + cost_values[val]
        time_sum = time_sum + time_values[val]
        cost_sum_dict[hyperparams[val]] = cost_sum
        time_sum_dict[hyperparams[val]] = time_sum
    elif hyperparams[val] != compare_hp:
        cost_sum_dict[hyperparams[val]] = cost_sum
        time_sum_dict[hyperparams[val]] = time_sum
        compare_hp = hyperparams[val]
        cost_sum = cost_values[val]
        time_sum = time_values[val]

16


In [57]:
def cma_algo():
    cma_list = list()
    cma_time_list = list()
    cma_dict_table = {"Algorithm": "CMA", "popsize": "na", "w": "na", "Cg": "na", "Cp": "na"}
    for k in range(0,  reps):
        cma_start_time = time.time()
        print("Start Time: ", cma_start_time)
        initial = np.zeros(nsupermarkets*2)
        sigma = 0.5

        estimate_cma = cma.CMAEvolutionStrategy(initial, sigma, {'bounds': [0.0, 1.0], 'seed': (k+1)})
        count = 0
        while not estimate_cma.stop() and count < maxits / estimate_cma.popsize:
            X = estimate_cma.ask()
            estimate_cma.tell(X, [total_travel_time(x.reshape(nsupermarkets,2),c,p) for x in X])#Solution
            estimate_cma.disp()
            count += 1

        xbest, cost_best, evals_best, evaluations, iterations, xfavorite, stds, stop = estimate_cma.result
        cma_list.append(cost_best)
        cma_end_time = time.time()
        print("End Time: ", cma_end_time)
        cma_time_list.append(cma_end_time - cma_start_time)
        
    cma_dict_table["min"] = min(cma_list)
    cma_dict_table["mean"] = np.mean(cma_list)
    cma_dict_table["stddev"] = np.std(cma_list)
    cma_dict_table["time"] = np.mean(cma_time_list)
    
    return cma_list

In [58]:
def main():
    main_start_time = time.time()
    print("main start time", main_start_time)
    rs_best = random_search_algo()
    pso_best = pso_algo()
    cma_best = cma_algo()
    main_end_time = time.time()
    print("main end time", main_end_time)
    
    print("Total Time = ", main_end_time - main_start_time)

In [59]:
if __name__ == "__main__":
    main()

main start time 1637439678.03729
Start Time:  1637439678.03729
End Time:  1637439678.549799
Start Time:  1637439678.549799
End Time:  1637439679.0294218
Start Time:  1637439679.0294218
End Time:  1637439679.5046966
Start Time:  1637439679.5056937
End Time:  1637439680.0033634
Start Time:  1637439680.0043604


2021-11-20 20:21:20,478 - pyswarms.single.global_best - INFO - Optimize for 250 iters with {'c1': 1.9, 'c2': 1.7, 'w': 0.9}


End Time:  1637439680.4731364
Start Time:  1637439680.4731364


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|250/250, best_cost=4.21e+5
2021-11-20 20:21:21,343 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 421016.4805773466, best pos: [-5.34296721  0.45729605  0.63005831  0.40022367  1.43451403 -2.36241105
  0.83288006  0.77272662  0.14345161  0.54639671]
2021-11-20 20:21:21,351 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 1.9, 'c2': 1.7, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|200/200, best_cost=5.14e+5
2021-11-20 20:21:22,141 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 513790.501718807, best pos: [ 0.63943665  0.40915032  0.50980902  0.6333211   0.76051087  0.83970722
 -2.00881197  1.4625664   1.01358906  0.73824222]
2021-11-20 20:21:22,146 - pyswarms.single.global_best - INFO - Optimize for 250 iters with {'c1': 1.9, 'c2': 1.7, 'w': 0.5}
pyswarms.sing

End Time:  1637439695.2203896
Start Time:  1637439695.2203896


...

End Time:  1637439722.2380486
Start Time:  1637439722.2380486


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|250/250, best_cost=4.95e+5
2021-11-20 20:22:03,031 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 494604.34287160134, best pos: [0.81986105 0.8040144  0.66479772 0.42179235 0.09638534 1.91218832
 0.06093699 0.24410917 0.66954873 1.86386445]
2021-11-20 20:22:03,036 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 1.9, 'c2': 1.7, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|200/200, best_cost=4.55e+5
2021-11-20 20:22:03,786 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 454956.6067867214, best pos: [0.41496124 0.97955981 0.64326579 0.40216383 0.6424747  0.1511158
 0.73332877 0.88973009 0.56258748 0.48597304]
2021-11-20 20:22:03,791 - pyswarms.single.global_best - INFO - Optimize for 250 iters with {'c1': 1.9, 'c2': 1.7, 'w': 0.5}
pyswarms.single.global_best: 100

End Time:  1637439735.6351109
Start Time:  1637439735.6351109


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|250/250, best_cost=5.24e+5
2021-11-20 20:22:16,416 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 524226.5169430863, best pos: [0.68741119 0.06085904 0.92692848 0.95626424 0.47678567 0.57698624
 0.76275617 0.65336587 0.68531612 0.38634862]
2021-11-20 20:22:16,422 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 1.9, 'c2': 1.7, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|200/200, best_cost=5.04e+5
2021-11-20 20:22:17,206 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 504461.11114154564, best pos: [ 0.44294481  0.36241608  0.63609695  0.37795341 -5.85023604  1.18291033
  0.50249149  0.58546981  0.75818773  0.72514606]
2021-11-20 20:22:17,213 - pyswarms.single.global_best - INFO - Optimize for 250 iters with {'c1': 1.9, 'c2': 1.7, 'w': 0.5}
pyswarms.single.globa

End Time:  1637439748.8099372
Start Time:  1637439748.8099372
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1, Sat Nov 20 20:22:28 2021)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 9.017699678172139e+05 1.0e+00 4.76e-01  5e-01  5e-01 0:00.0
    2     20 7.418654975188125e+05 1.2e+00 4.48e-01  4e-01  5e-01 0:00.0
    3     30 8.851369320176207e+05 1.2e+00 4.73e-01  4e-01  5e-01 0:00.0
  100   1000 4.277889696570773e+05 1.0e+01 1.45e-01  3e-02  2e-01 0:00.2
  200   2000 2.433140098113324e+05 3.3e+01 1.65e-03  7e-05  2e-03 0:00.3
  300   3000 2.418475420343096e+05 3.1e+01 8.34e-06  1e-07  4e-06 0:00.5
  400   4000 2.418457515038924e+05 4.6e+01 1.93e-08  1e-10  4e-09 0:00.6
  499   4990 2.418457491827580e+05 6.2e+01 8.10e-11  2e-13  1e-11 0:00.8
End Time:  1637439749.7034335
Start Time:  1637439749.7044308
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=2, Sat Nov 20 20:22:29 2021)
Iterat #Fevals   function value  axis ratio  s